# Polygons

## The Problem

Consider an n-sided polygon. Drawing all the lines that connect the vertices, mark out the intersection points. Define a "decreasing path" as a path from the starting point to the end point, where the distance to the end point is only decreasing. Using only the vertices of the polygon and the intersection points between these lines, find the total number of decreasing paths.

## Solution Approach

1. Draw a regular polygon
2. Draw all lines between points and find the coordinates of all the intersection points
3. Draw a circle that is anchored at the endpoint, and wraps the octagon neatly; the diameter of this circle should start at L2 distance of [start point - end point]
4. Slowly reduce the circle size. It appears that points that fall out of the circle first have arrows that draw to points that fall out of the circle later

In [1]:
import numpy as np
import itertools
from math import sin, cos, pi
from typing import List, Tuple

### Create the regular polygon
Let's define some restrictions on the polygon.
1. Centred on (0,0) with radius 1 unit; thus start point is (1,0) and end point is (-1,0)
2. Use the internal angle of a polygon to find the points of the vertices

See https://en.wikipedia.org/wiki/Rotation_matrix.

Some of the code for Line and Point classes are adapted from https://stackoverflow.com/a/20679579.

In [2]:
# Rounding precision for floating point calculations
PRECISION = 14

In [3]:
class Point:
    def __init__(self, x: float, y: float):
        self.raw_x = x
        self.raw_y = y
        self.x = round(x, PRECISION) if x != None else None
        self.y = round(y, PRECISION) if y != None else None
        
    @classmethod
    def from_intersection(cls, L1: "Line"=None, L2: "Line"=None) -> "Point":
        D  = L1.a * L2.b - L1.b * L2.a
        Dx = L1.c * L2.b - L1.b * L2.c
        Dy = L1.a * L2.c - L1.c * L2.a
        
        if D != 0:
            x = round(Dx / D, PRECISION)
            y = round(Dy / D, PRECISION)
            
            # Disregard points outside the polygon circle
            if abs(x) > 1 or abs(y) > 1:
                x, y = None, None
        else:
            x, y = None, None
        
        return cls(x, y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __hash__(self):
        return hash((self.x, self.y))


In [4]:
class Line:
    def __init__(self, p1, p2):
        self.a = p1.y - p2.y
        self.b = p2.x - p1.x
        self.c = p1.x*p2.y - p2.x*p1.y
        
    def __repr__(self):
        return f"Line({self.a}, {self.b}, {self.c})"

In [5]:
def find_next_coordinate(xy: Tuple[float, float], t: float):
    x, y = xy
    return (x*cos(t) - y*sin(t), x*sin(t) + y*cos(t))

def create_regular_polygon(n: int) -> List[Tuple]:
    # Defines a polygon as the list of (x,y) coordinates of the vertices
    theta = 2/n * pi
    polygon = []
    start_xy = (1,0)
    
    current_xy = start_xy
    for i in range(n):
        current_xy = find_next_coordinate(current_xy, theta)
        polygon.append(current_xy)
    polygon = [Point(x,y) for x,y in polygon]
    return polygon

In [6]:
# Test the polygon creation function
create_regular_polygon(6)

[Point(0.5, 0.86602540378444),
 Point(-0.5, 0.86602540378444),
 Point(-1.0, 0.0),
 Point(-0.5, -0.86602540378444),
 Point(0.5, -0.86602540378444),
 Point(1.0, -0.0)]

### Find the intersection points

In [7]:
polygon = create_regular_polygon(6)
lines = [Line(a,b) for a,b in list(itertools.combinations(polygon, 2))]
points = set(Point.from_intersection(a,b) for a,b in list(itertools.combinations(lines, 2)))

In [8]:
polygon

[Point(0.5, 0.86602540378444),
 Point(-0.5, 0.86602540378444),
 Point(-1.0, 0.0),
 Point(-0.5, -0.86602540378444),
 Point(0.5, -0.86602540378444),
 Point(1.0, -0.0)]

In [9]:
lines

[Line(0.0, -1.0, 0.86602540378444),
 Line(0.86602540378444, -1.5, 0.86602540378444),
 Line(1.73205080756888, -1.0, 0.0),
 Line(1.73205080756888, 0.0, -0.86602540378444),
 Line(0.86602540378444, 0.5, -0.86602540378444),
 Line(0.86602540378444, -0.5, 0.86602540378444),
 Line(1.73205080756888, 0.0, 0.86602540378444),
 Line(1.73205080756888, 1.0, 0.0),
 Line(0.86602540378444, 1.5, -0.86602540378444),
 Line(0.86602540378444, 0.5, 0.86602540378444),
 Line(0.86602540378444, 1.5, 0.86602540378444),
 Line(0.0, 2.0, 0.0),
 Line(0.0, 1.0, 0.86602540378444),
 Line(-0.86602540378444, 1.5, 0.86602540378444),
 Line(-0.86602540378444, 0.5, 0.86602540378444)]

In [10]:
points.remove(Point(None, None))

In [11]:
points

{Point(-0.25, -0.43301270189222),
 Point(-0.25, 0.43301270189222),
 Point(-0.5, -0.28867513459481),
 Point(-0.5, -0.86602540378444),
 Point(-0.5, 0.0),
 Point(-0.5, 0.28867513459481),
 Point(-0.5, 0.86602540378444),
 Point(-1.0, 0.0),
 Point(0.0, -0.57735026918963),
 Point(0.0, 0.0),
 Point(0.0, 0.57735026918963),
 Point(0.25, -0.43301270189222),
 Point(0.25, 0.43301270189222),
 Point(0.5, -0.28867513459481),
 Point(0.5, -0.86602540378444),
 Point(0.5, 0.0),
 Point(0.5, 0.28867513459481),
 Point(0.5, 0.86602540378444),
 Point(1.0, 0.0)}

### Get all the line segments

### Draw a circle that is anchored at the endpoint and wraps the octagon neatly.